# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [87]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [88]:
# checking the current working directory
print(os.getcwd())

# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [73]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [74]:
# check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821



- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [75]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [76]:
# Creating a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS cassandra_etl
    WITH REPLICATION = 
    {'class': 'SimpleStrategy', 'replication_factor': 1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [77]:
# Setting KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('cassandra_etl')
except Exception as e:
    print(e)

## Query 1: Giving the artist, song title and song length that was heard in sessionId 338 and itemInSession were 4

In [78]:
## Query 1: Give the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query_1 = "CREATE TABLE IF NOT EXISTS song_details (sessionId text, itemInSession text, artist text, song_title text, song_length text, PRIMARY KEY(sessionId, itemInSession))"
session.execute(query_1)

In [80]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assigning the INSERT statements into the `query` variable
        query = "INSERT INTO song_details (sessionID, itemInSession, artist, song_title, song_length)"
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        ## Assigning which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[8], line[3], line[0], line[-2], line[5]))

In [84]:
## Adding in the SELECT statement to verify the data was entered into the table
x = session.execute("SELECT artist, song_title FROM song_details WHERE sessionId = '338'")
for i in x:
    print(i.artist)

Pixies
The Roots / Jack Davey
Mike And The Mechanics
Faithless


In [83]:
## QUERY 1
q = "SELECT artist, song_title, song_length FROM song_details WHERE sessionId = '338' and itemInSession = '4'"
rows = session.execute(q)
for row in rows:
    print(row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.3073


## Query 2: Giving only the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [61]:
query_2 = "CREATE TABLE IF NOT EXISTS song_playlist_session\
(userid text, sessionid text, itemInSession text, artist text, song text, fname text, lname text,\
            PRIMARY KEY((userId, sessionId), itemInSession))"
try:
    session.execute(query_2)
except Exception as e:
    print(e)

In [62]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:    
        query = "INSERT INTO song_playlist_session (userid, sessionid, itemInSession, artist, song, fname, lname)"
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
        
        session.execute(query, (line[-1], line[8], line[3], line[0], line[-2], line[1], line[4]))

In [63]:
q2 = "SELECT artist, song, fname, lname FROM song_playlist_session WHERE userid = '10' AND sessionid = '182'"
rows = session.execute(q2)
for row in rows:
    print(row.artist, row.song, row.fname, row.lname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Query 3: Giving the first and last name of users who listened to 'All Hands Against His Own' song

In [64]:
q = "CREATE TABLE IF NOT EXISTS user_to_song (song text, userid text, fname text, lname text, PRIMARY KEY (song, userid))"
session.execute(q)

In [65]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:    
        query = "INSERT INTO user_to_song (song, userid, fname, lname)"
        query = query + "VALUES(%s, %s, %s, %s)"
        
        session.execute(query, (line[-2], line[-1], line[1], line[4]))

In [66]:
## QUERY 3
q3 = "SELECT fname, lname FROM user_to_song WHERE song = 'All Hands Against His Own'"
s = session.execute(q3)
for i in s:
    print(i.fname, i.lname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [85]:
q1 = "DROP TABLE song_details"
q2 = "DROP TABLE song_playlist_session"
q3 = "DROP TABLE user_to_song"

session.execute(q1)
session.execute(q2)
session.execute(q3)

InvalidRequest: Error from server: code=2200 [Invalid query] message="unconfigured table song_playlist_session"

### Close the session and cluster connection¶

In [86]:
session.shutdown()
cluster.shutdown()